## D'NOVA YouTube Shorts Comments collection
- selenium을 통해 쇼츠 커멘트 수집
- 기본적으로 shorts url은 전부 가지고 있다고 가정

In [8]:
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [1]:
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def random_sleep():
    """ 1~2초 사이의 랜덤한 대기 """
    time.sleep(random.uniform(1, 2))

def scroll_to_bottom(driver):
    """ 페이지를 끝까지 스크롤 다운하는 함수 """
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        
        # 스크롤 다운 후 로딩 시간을 추가 (1~2초 사이의 랜덤 대기)
        time.sleep(random.uniform(0.5, 1))
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_youtube_comments(url):
    # uc driver로 브라우저 실행
    driver = uc.Chrome()
    driver.get(url)
    random_sleep()  # 페이지가 로드될 시간을 줍니다.
    
    # 페이지 끝까지 스크롤 다운
    scroll_to_bottom(driver)
    
    # 모든 댓글 수집
    comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
    comments = [comment.text for comment in comments_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 댓글 작성 날짜 수집
    date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
    dates = [date.text for date in date_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 결과를 리스트의 튜플 형태로 반환
    comments_data = list(zip(comments, dates))
    
    # 드라이버 종료
    driver.quit()
    
    return comments_data


In [23]:
# 예시 URL
url = "https://www.youtube.com/watch?v=xEuH_5Ik92U"
comments_list = get_youtube_comments(url)

# 리스트와 튜플 형태의 데이터 출력
print(len(comments_list))

246


#### Block 될 경우 cookie를 사용해 login 후 수집 진행

In [1]:
import sys
import pandas as pd
import time
import pickle
import os
import re
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# Define the range for the random sleep time in seconds
min_sleep_time, max_sleep_time = 1, 3
def random_sleep():
    """ 1~2초 사이의 랜덤한 대기 """
    time.sleep(random.uniform(1, 2))

def scroll_to_bottom(driver):
    """ 페이지를 끝까지 스크롤 다운하는 함수 """
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        
        # 스크롤 다운 후 로딩 시간을 추가 (1~2초 사이의 랜덤 대기)
        time.sleep(random.uniform(0.5, 1))
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def save_progress(df, output_file):
    """ 현재 진행 상태를 엑셀 파일로 저장하는 함수 """
    df.to_excel(output_file, index=False)
    print(f"Progress saved to {output_file}")

# Function to create ChromeOptions and a new Chrome driver
def create_driver():
    """Create a new instance of Chrome driver with fresh ChromeOptions."""
    options = uc.ChromeOptions()
    options.headless = False
    options.add_experimental_option('prefs', {'intl.accept_languages': 'en'})
    
    driver = uc.Chrome(options=options, use_subprocess=True)
    driver.set_window_size(1000, 800)
    return driver

initial_path = ""
cookie_files = [
    "cookies_list/cookies_hyunjun960214.pkl",
    "cookies_list/cookies_ytcollect1.pkl",
    "cookies_list/cookies_yttrendreport.pkl"
]
cookie_index = 0

def login(driver):
    """Login to YouTube using the provided driver and rotating cookie files."""
    global cookie_index
    cookies_file_path = initial_path + cookie_files[cookie_index]
    print(f"Using cookies from: {cookies_file_path}")

    if os.path.exists(cookies_file_path):
        with open(cookies_file_path, 'rb') as cookies_file:
            cookies = pickle.load(cookies_file)
            driver.get('https://www.youtube.com')
            driver.implicitly_wait(10)
            for cookie in cookies:
                driver.add_cookie(cookie)
            print("Cookies added")
            driver.refresh()
            driver.implicitly_wait(10)
    else:
        driver.get('https://www.youtube.com')
        driver.implicitly_wait(15)
        try:
            clk = driver.find_element(By.XPATH, "(//div[@class='yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response-inverse'])[2]").click()
            time.sleep(random.uniform(1, 3))
        except:
            pass
        try:
            driver.find_element(By.XPATH, "//*[@id='buttons']/ytd-button-renderer/yt-button-shape/a/yt-touch-feedback-shape/div").click()
            time.sleep(random.uniform(1, 3))
            email_field = driver.find_element(By.XPATH, "//*[@id='identifierId']")
            email_field.send_keys("yttrendreport")
            time.sleep(random.uniform(1, 3))
            email_field.send_keys(Keys.ENTER)
            time.sleep(random.uniform(1, 3))
            email_pass = driver.find_element(By.XPATH, "//*[@id='password']/div[1]/div/div[1]/input")
            email_pass.send_keys("hjl960214!!")
            time.sleep(1)
            email_pass.send_keys(Keys.ENTER)
            time.sleep(random.uniform(1, 3))
            driver.implicitly_wait(10)
        except:
            pass
        with open(cookies_file_path, 'wb') as cookies_file:
            pickle.dump(driver.get_cookies(), cookies_file)
        print("Cookies saved")

    cookie_index = (cookie_index + 1) % len(cookie_files)

def process_youtube_comments(driver):
    """Process YouTube video comments and save progress whenever an error occurs or the program ends."""    
    try:
        # Load top 100 rows from the Excel file
        loaded_df_100 = pd.read_excel("loaded_db_100.xlsx")
        print(loaded_df_100.head())
        
        for index, row in loaded_df_100.iterrows():
            time.sleep(random.uniform(1, 2))
            title = None  # Initialize title here to avoid 'referenced before assignment' error
            
            try:
                url = row['shorts_url']
                title = row['channel_name']  
                driver.get(url)
                driver.implicitly_wait(10)         
                # Scroll down to load comments
                scroll_to_bottom(driver)
                
                # Get comments and dates
                comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
                comments = [comment.text for comment in comments_elements]
                random_sleep() 
                
                date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
                dates = [date.text for date in date_elements]
                random_sleep()  
                
                # Pair comments and dates, or set as NaN if no comments were found
                comments_data = list(zip(comments, dates)) if comments else [(None, None)]
                print(len(comments_data), comments_data)
                # Store in DataFrame
                loaded_df_100.at[index, 'shorts_comments_time'] = [comments_data]

            except Exception as err:
                print(f"Error processing {title or 'Unknown'} at row {index}: {err}")
                output_file = f"loaded_df_100.xlsx"
                save_progress(loaded_df_100, output_file)
                raise 

    except Exception as e:
        print(f"Error processing file: {e}")
        raise 

    finally:
        output_file = f"loaded_df_100.xlsx"
        save_progress(loaded_df_100, output_file)
        print(f"Progress saved for file: {output_file}")
        
def execute_crawling():
    try:
        driver = create_driver()
        login(driver=driver)
        process_youtube_comments(driver)
        print(f"Process Waiting...")
        time.sleep(random.uniform(1, 2))
    except Exception as e:
        print(f"Error occurred while processing : {e}. Restarting with a new cookie.")
        
        if driver:
            driver.quit()
        
        time.sleep(random.uniform(120, 150))  
        driver = create_driver()  
        login(driver=driver)
        try:
            process_youtube_comments(driver)
        except Exception as retry_error:
            print(f"Failed again while processing : {retry_error}")

    finally:
        if driver:
            driver.quit()

# Execute the crawling process
execute_crawling()


Using cookies from: cookies_list/cookies_hyunjun960214.pkl
Cookies added
                            channel_name  \
0                            Marc Brunet   
1                               Jen_ny69   
2                           DawateIslami   
3  Go Natural English with Gabby Wallace   
4                   ReligionForBreakfast   

                                    shorts_url shorts_title  \
0  https://www.youtube.com/watch?v=Qc6NO19u3MM          NaN   
1  https://www.youtube.com/watch?v=ixq_pOK7jk0          NaN   
2  https://www.youtube.com/watch?v=JhocFEigq3k          NaN   
3  https://www.youtube.com/watch?v=v9dcf7OF1Wg          NaN   
4  https://www.youtube.com/watch?v=WZn6dF9C1NQ          NaN   

                                  shorts_description  \
0  👨‍🎨 Serious about your art? Check out my ART S...   
1          Watch full video on ‪@adayinthelifeofa69‬   
2  Kabhi Ye Bhi Try Karen | #dawateislami #whatsa...   
3                                                NaN   
4  

C:\Users\T14Gen1\AppData\Local\Temp\ipykernel_8412\3228636942.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[('The audacity of assuming I can draw those perfect boxes.', '2 days ago (edited)'), ('That’s the quickest little gem of anatomy tips I’ve ever seen…!', '2 days ago'), ("When I practice I can do it, but when I'm drawing I seem to forget everything.", '2 days ago'), ('GOD YOU LITERALLY SOLVED EVERY (AND GOD SHOULD CURSE ME IF I LIE) EVERY SINGLE PROBLEM I STRUGGLED WITH IN FEW SECONDS', '2 days ago'), ('Bro, you make it look so easy..', '1 day ago'), ('Yep 100% box method helped me I was struggling with basic human design for years and just in 5 minutes I can finally draw human characters now I just need to learn shading', '1 day ago'), ('Thanks Professor Xavier', '12 hours ago'), ('Bro i love you ima watch all your videos and gets your course immediately, rn i’m following the 30 days chall

Jen_ny69
https://www.youtube.com/watch?v=ixq_pOK7jk0
DawateIslami
https://www.youtube.com/watch?v=JhocFEigq3k
Go Natural English with Gabby Wallace
https://www.youtube.com/watch?v=v9dcf7OF1Wg
ReligionForBreakfast
https://www.youtube.com/watch?v=WZn6dF9C1NQ
Medicosis Perfectionalis
https://www.youtube.com/watch?v=wFVlzZUWFcw
理科太太 Li Ke Tai Tai
https://www.youtube.com/watch?v=E1789fuBkkU
Brian Tracy
https://www.youtube.com/watch?v=C6HaBSkA6xU
Now You See It
https://www.youtube.com/watch?v=g6lcT4EhNzM
Progress saved to loaded_df_100.xlsx
Progress saved for file: loaded_df_100.xlsx
Process Waiting...
